In [2]:
import pandas as pd
import math
import logging
from sklearn.model_selection import TimeSeriesSplit
from copy import deepcopy
from prophet import Prophet
from prophet.serialize import model_to_json


from eval import my_grid_search_cv

# from tqdm import tqdm
# tqdm.pandas()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   store_id                        252108 non-null  object        
 1   visit_date                      252108 non-null  datetime64[ns]
 2   visitors                        252108 non-null  int64         
 3   day_of_week                     252108 non-null  object        
 4   holiday_flg                     252108 non-null  int64         
 5   genre_name                      252108 non-null  object        
 6   area_name                       252108 non-null  object        
 7   latitude                        252108 non-null  float64       
 8   longitude                       252108 non-null  float64       
 9   year                            252108 non-null  int64         
 10  month                           252108 non-null  int64  

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

In [ ]:
data = pd.read_csv("data/processed/data.csv")
data.head()

data["date"] = pd.to_datetime(data["date"])
data = data.rename(columns={"date": "visit_date"})

data.info()

In [ ]:
data = data[data["store_id"] == "air_00a91d42b08b08d9"]

X = data.drop(columns=["visitors", "visit_date"]).reset_index(drop=True)
y = data[["visitors"]].reset_index(drop=True)
# X



In [ ]:
import lightgbm as lgb
param_grid = {
    "max_depth": [3, 4, 6, 5, 10],
    "num_leaves": [10, 20, 30, 40, 100, 120],
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "n_estimators": [50, 100, 300, 500, 700, 900, 1000],
    "colsample_bytree": [0.3, 0.5, 0.7, 1],
    "device": ["gpu"]
}
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c] = X[c].astype('category')

params = my_grid_search_cv(lgb.LGBMRegressor, X, y, param_grid, n_jobs=-1)
model = lgb.LGBMRegressor(**params)
model.fit(X, y)